In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh,eigs
from scipy.misc import comb
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct
sym.init_printing()

from IPython.display import display

# The Preisach transition operator

We consider a discretized version of a Preisach system.  The current state is represented by a sequence of zeros and ones of length $N$, which indicate horizontal and vertical sections in the usual staircase representation discretized on a regular grid. Example: 
\begin{equation}
Q = 10010011
\end{equation}
The input acts from the right hand end and goes either up or down. If the input goes up the effect is that a one gets "pushed in" from right to left onto the first zero bit, i.e if the input goes up, the above state gets converted into 
\begin{equation}
Q \to Q^+ = 10010111
\end{equation}
Similarly input step down inserts a zero from the right, i.e. 
\begin{equation}
Q \to Q^- = 10010010
\end{equation}
Depending on the sequence of inputs we therefore arrive at different states which can be calculated. 

In the simplest case the inputs are just going randomly up or down at each discrete timestep with equal probability.  This means that we can construct a transition matrix $T$ for this Markov chain as follows. For $N=1$ there are only two possible states, either 0 or 1. starting from state zero, if the input goes up we arrive at state 1. However if we are in state 0 and the input goes down, we leave the current state space and the process stops. We can therefore think of the system passing an absorbing boundary if this happens. Therefore if we start at state zero, we make a transition to 1 with probability $1/2$ or get absorbed at the boundary with probability $1/2$. Similarly for starting at state 1.  If $p(0,t)$ and $p(1,t)$ are the probabilities of states 0 or 1 at time $t$ we can write
\begin{equation}
\left(\begin{array}{c}
p(0,t+1)\\
p(1,t+1)
\end{array}\right)
= \frac{1}{2} \left(\begin{array}{cc}
0 & 1\\
1 & 0 
\end{array}\right)
\left(\begin{array}{c}
p(0,t)\\
p(1,t)
\end{array}\right)
\end{equation}
This thus defines the transition matrix 
\begin{equation}
T_1= \frac{1}{2} \left(\begin{array}{cc}
0 & 1\\
1 & 0 
\end{array}\right)
\end{equation}

We can now go on to define $T_N$ recursively from the knowledge of $T_{N-1}$ by noting that $T_N$ mostly acts on the last $N-1$ elements in the same way as $T_{N-1}$ irrespectively of the value of the first element. The only exception concerns the two states $100\cdots 00$ and $011\cdots 11$ which now with probability $1/2$ get mapped to states $000\cdots 00$ and $111\cdots 11$, respectively. More formally, we define 
\begin{equation}
T_{N}=  \left(\begin{array}{c|c}
T_{N-1} & 
\begin{array}{c|ccc}
\frac{1}{2} & 0 & \cdots & 0  \\
\hline
0 & 0 & \cdots & 0  \\
\vdots &\vdots & \ddots & \vdots  \\
0& 0 & \cdots & 0  \\
\end{array}\\ 
\hline
\begin{array}{ccc|c}
 0 & \cdots & 0 & 0 \\
\vdots & \ddots & \vdots &\vdots \\
0 & \cdots & 0 & 0 \\
\hline
0 & \cdots & 0 &\frac{1}{2}  \\
\end{array} & T_{N-1} 
\end{array}\right)
\end{equation}

To show some explicit examples:

In [ ]:
def TP(N):
    I2 = sym.Matrix.eye(2)

    T = sym.Matrix.zeros(1)

    for i in range(1,N+1):
        T = TensorProduct(I2, T)
        T[2**(i)-1,2**(i-1)-1] = 1
        T[0,2**(i-1)] = 1
        
    return T / 2

for i in range(4):
    display(i,2*TP(i))

In general, the system is highly degenerate. We have the following symmetry:

- $S_p$ which exchanges every 0 with a 1 and vice versa. This means that the $N$ digit binary number $Q$ gets translated into $2^N-1-Q$.

Let us study the eigenvalues and eigenvectors of the first couple of $T_P$ explicitly. Let us use the notation

\begin{equation}
T_1 =\frac{1}{2} \sigma_x =  \frac{1}{2}\left( \begin{array}{cc} 0 & 1 \\ 1 & 0 \end{array} \right)
\end{equation}
which obviously has eigenvalues $-1/2, +1/2$ and eigenvectors
\begin{align}
|-\rangle &= |0\rangle - |1\rangle\\ 
|+\rangle &= |0\rangle + |1\rangle\\ 
\end{align}
It is also useful to reverse these relationships
\begin{align}
|0\rangle &= \frac{1}{2} \left[|-\rangle + |+\rangle \right] \\
|1\rangle &= \frac{1}{2} \left[-|-\rangle + |+\rangle \right]
\end{align}

Next we can write 
\begin{equation}
T_2 = I \otimes T_1 + |0\rangle \otimes |0\rangle \langle 1 | \otimes \langle 0 | + |1\rangle \otimes |1\rangle \langle 0 | \otimes \langle 1 | 
\end{equation}


This has the eigenvalues

In [ ]:
TP(2).eigenvects()

Or in more mathematical vector notation we have 
\begin{align}
\lambda \in \left\{{0,0,-\frac{1}{\sqrt{2}},\frac{1}{\sqrt{2}}}\right\}
\end{align}
\begin{align}
|v_0\rangle &= |0\rangle \otimes |1\rangle - |1\rangle \otimes |0\rangle \\
|v_-\rangle &= \left[|0\rangle \otimes |0\rangle + |1\rangle \otimes |1\rangle\right] - \frac{1}{\sqrt{2}}\left[|0\rangle \otimes |1\rangle + |1\rangle \otimes |0\rangle\right] \\
|v_+\rangle &= \left[|0\rangle \otimes |0\rangle + |1\rangle \otimes |1\rangle\right] + \frac{1}{\sqrt{2}}\left[|0\rangle \otimes |1\rangle + |1\rangle \otimes |0\rangle\right] \\
\end{align}

For the degenerate Jordan Block we find

In [ ]:
TP(2)**2, TP(2) * sym.Matrix([[2],[0],[0],[-2]])

Which means that we can identify 
\begin{equation}
|v_p\rangle = 2 \left[ |0\rangle \otimes |0\rangle - |1\rangle \otimes |1\rangle \right]
\end{equation}

Thus in the basis $|v_0\rangle$, $|v_p\rangle$,$|v_-\rangle$,$|v_+\rangle$ the matrix $T_2$ has the form
\begin{equation}
T_2 = \left( \begin{array}{cccc} 0 & 1 & 0 & 0 \\
                                 0 & 0 & 0 & 0 \\ 
                                0 & 0 & -\frac{1}{\sqrt{2}} & 0 \\
                                0 & 0 & 0 &  \frac{1}{\sqrt{2}} \\
                                \end{array}\right)
\end{equation}


In [ ]:
ev3 = TP(3).eigenvects()
ev3[0], ev3[1]

In [ ]:
ev3[2], ev3[3][:2], sym.simplify(ev3[3][2][0]), ev3[4][:2], sym.simplify(ev3[4][2][0]), sym.simplify(ev3[4][2][0]+ev3[3][2][0]), 

For $T_3$ we find the eigenvalues $\lambda \in \left\{ -\frac{1}{2}, 0, 0 \right\} $
and eigenvectors
\begin{align}
|v_{-1/2}\rangle &= |010\rangle - |011\rangle - |100\rangle + |101\rangle = |01\rangle \otimes |v_{-1}\rangle - |10\rangle \otimes |v_{-1}\rangle = \left(|01\rangle - |10\rangle \right) \otimes |v_-\rangle \\
|v_{01}\rangle &= |001\rangle - |010\rangle \\
|v_{01}\rangle &= |101\rangle - |110\rangle \\
|v_{1/2}\rangle &= |010\rangle + |011\rangle - |100\rangle - |101\rangle = \left(|01\rangle - |10\rangle \right) \otimes |v_+\rangle  \\
\end{align}


In [ ]:
x = sym.symbols("x")
sym.factor(x**4-1)

In [ ]:
def Permutation(sigma):
    n = len(sigma)
    result = sym.Matrix.zeros(n)
    for i,j in enumerate(sigma):
        result[i,j] = 1
    return result

def SP(n):
    sigma_x = sym.Matrix([[0,1],[1,0]])
    T=sym.Matrix([[1]])
    for i in range(n):
        T = TensorProduct(sigma_x, T)
    return T

def PM(n):
    sigma_xp = sym.Matrix([[1,1],[-1,1]])/sym.sqrt(2)
    T=sym.Matrix([[1]])
    for i in range(n):
        T = TensorProduct(sigma_xp, T)
    return T

def PM2(n):
    result = sym.Matrix.eye(2**n)
    for i in range(2**(n-1)):
        result[i, -(i+1)] =  1
        result[-(i+1), i] = -1
    return result/sym.sqrt(2)

Permutation([1,2,0,3])

In [ ]:
TensorProduct(sym.Matrix.eye(2), PM2(2))

In [ ]:
P = Permutation([0, 6, 5,3, 4,2, 1, 7])
H = TensorProduct(sym.Matrix.eye(2), PM2(2))
TPSQ = TP(3) @ TP(3)
P @ TP(3) @ P.T, PM2(3).T @ TPSQ @ PM2(3), TPSQ, PM2(3), PM2(3).T @ P @ TPSQ @ P.T @ PM2(3)


In [ ]:
P = Permutation([0, 1,2,4,8, 3,5,6,9,10,12, 7,11,13,14, 15])
P = Permutation([0,  3,5,6,9, 10,12, 15, 1,2,4,7,8,11,13,14])
P = Permutation([0,  3, 6, 5, 1, 2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15 ])
Q = TP(4)@ TP(4)
H1 = PM2(4) @ P.T # P.T  @ TensorProduct(sym.Matrix.eye(2), PM2(3))
H = H1 @ TensorProduct(sym.Matrix.eye(2**2), PM2(2)) #@ TensorProduct(
    #sym.Matrix.eye(2**3), PM2(1))
Q, H1.T @ Q @ H1, 

In [ ]:
D1 =  PM2(4)@ TP(4) @ PM2(4).T
T = D1[:2**3,:2**3]
P = Permutation([0,3,5,6,1,2,4,7])
T, P @ T  @ P.T, P @ T @ T @ P.T

In [ ]:
(P @ T @ T @ P.T).eigenvects()

In [ ]:
Q = sym.Matrix.eye(2**3)
Q[0,-1]=1
Q[-1,0]=-1
Q @ PM2(3) @ T @ PM2(3).T @ Q.inv()
T.eigenvects(), T.T.eigenvects()

In [ ]:
T = TP(5)
lmes =  T.eigenvects()
for lam, mult, evs in lmes:
    print("{:20} {} {}".format(float(sym.acos(lam)/sym.pi), mult, len(evs)))
    

In [ ]:
T = TP(1)
for lam, mult, evs in T.eigenvects():
    print("{} {} {}".format(sym.acos(lam)/sym.pi, mult, len(evs)))
    

In [ ]:
#Q3 =TP(3).eigenvects()
#Q4 =TP(4).eigenvects()
for lam, mult, evs in Q3:
    print(sym.simplify(sym.acos(lam)/sym.pi), mult,len(evs))
    
print()
for lam, mult, evs in Q4:
    print(sym.simplify(sym.acos(lam)/sym.pi), mult,len(evs))

In [ ]:
sym.simplify(Q3[3][2][0]),sym.simplify(Q3[4][2][0]),sym.simplify(Q3[5][2][0]),sym.simplify(Q3[6][2][0])

In [ ]:
sym.simplify(Q4[5][2][0]),sym.simplify(Q4[6][2][0]),sym.simplify(Q4[3][2][0]),sym.simplify(Q4[4][2][0])

In [ ]:
def chebyshev(n):
    result = [1,2*x]
    while len(result) < n:
        result.append(sym.simplify(2*x*result[-1] - result[-2]))
    return result

chebyshev(7)

In [ ]:
myx = np.linspace(-1,1,200)
for T in chebyshev(5)[1:]:
    T_np = sym.lambdify(x, T,"numpy")
    
    plt.plot(myx, T_np(myx))
plt.plot(myx,0*myx,"k")
plt.savefig("cheby_u.pdf")

The characteristic polynomials seem to have the form 
\begin{equation}
  \chi_n(\lambda) = U_{n+1}(\lambda)U_{n-1}(\lambda)U_{n-2}(\lambda)^2 
  U_{n-3}(\lambda)^4 \cdots U_{1}(\lambda)^{2^{n-2}}
\end{equation}
where $U_{n}(\lambda)$ are the Chebyshev polynomials of the second kind, defined through 
\begin{equation}
U_{n+1} = 2 \lambda U_{n} - U_{n-1}
\end{equation}

Let us check that this is true at least for a couple of $n$.  To do this define `chi(n,x)` and `chi_theory(n,x)` which calculate the characteristic polynomials directly and through the above formula:

In [ ]:
x = sym.symbols("x")
def cheby2(n,x):
    if n<0:
        return 0
    rold = 1
    if n==0:
        return rold
    r = 2*x

    
    for i in range(1,n):
        rnew = 2*x* r - rold
        rold =r
        r = rnew
    return r
    
def chi_theory(n,x):
    if n < 1:
        return 1
    result = cheby2(n+1, x) 
    for i in range(1,n):        
        result *= cheby2(i, x)**(2**(n-i-1))
    return result/2**2**n
    
def chi(n,x):
    return TP(n).charpoly(x)


Indeed the first couple of polynomials agree: 

In [ ]:
x = sym.symbols("x")

for i in range(1,6):
    display(i,chi(i,x), chi_theory(i,x).expand())

On the other hand it is known that the zeros of the Chebyshev polynomial $U_n$ are given by 
\begin{equation}
x_k = \cos\left(\pi \frac{k + 1}{n +1}\right) \qquad \text{for $k=0,\ldots, n-1$}
\end{equation}
Therefore the values and numbers of zeros of $\chi_n$ can be calculated as follows:


In [ ]:
def zeros_theory(n):
    if n < 1:
        return 1
    counter = defaultdict(int)
    for k in range(n+1):
            counter[sym.Rational(k+1,n+2)] += 1
        
    for i in range(1,n):
        for k in range(i):
            counter[sym.Rational(k+1,i+1)] += (2**(n-i-1))
    
    ks = list( counter.keys() )
    ks.sort(reverse=True)
    return [(sym.N(sym.cos(sym.pi * f )),counter[f]) for f in ks]


In [ ]:
def zeros_theory_frac(n):
    if n < 1:
        return 1
    counter = defaultdict(int)
    for k in range(n+1):
            counter[sym.Rational(k+1,n+2)] += 1
        
    for i in range(1,n):
        for k in range(i):
            counter[sym.Rational(k+1,i+1)] += (2**(n-i-1))
    
    ks = list( counter.keys() )
    ks.sort(reverse=True)
    return [ (f ,counter[f]) for f in ks]


In [ ]:
zeros_theory_frac(4)

In [ ]:
%matplotlib notebook
k = 0
r = []
pos = 0
N=20
for p in zeros_theory(N):
    r.append([pos,p[0]])
    pos += p[1]
    r.append([pos,p[0]])
    
r = np.array(r)
plt.plot(r[:,0]/2**N,r[:,1])
plt.xlim([0,1])
plt.ylim([-1,1])

plt.plot([0,1],[-1,1])
plt.xlabel("#")
plt.ylabel(r"$\lambda$")
plt.savefig("devil_N="+str(N)+".pdf")

Reproduce the function $f(x)$ of equation (2) in http://mathworld.wolfram.com/DevilsStaircase.html:
\begin{equation}
f(x) = \sum_{n=1}^\infty \frac{\lfloor nx \rfloor}{2^n} = \sum_{m=1}^\infty \frac{1}{2^{\lfloor m/x \rfloor}}
\end{equation}

In [ ]:
%matplotlib inline
def devil(x):
    res = 0
    for n in range(20):
        res = res + np.floor(n*x)/2**n
    return res

myx = np.linspace(0,1,1000)
myy = devil(myx)

plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.plot(myx, myy)
plt.savefig("f(x) mathworld.pdf")

In [ ]:
%matplotlib inline


myx = np.linspace(0,1,1000)
myy = devil(myx)

plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(myy, myx)

k = 0
r = []
pos = 0
N=20
for p in zeros_theory_frac(N):
    # Add a flat interval of length p[1]
    r.append([pos,p[0]])
    pos += p[1]
    r.append([pos,p[0]])
    
r = np.array(r)
plt.plot(r[:,0]/2**N,1-r[:,1])
plt.xlim([0,1])
plt.ylim([0,1])

plt.xlabel("#")
plt.ylabel(r"$\lambda$")
plt.savefig("devil_N="+str(N)+".pdf")

In [ ]:
N=14

Let's now directly calculate the spectrum numerically for some reasonably large $N$.

For $N=14$ we need about six hours of calculation time.  If we have calculated that once, we can also skip to the next cell and load the precomputed data.

In [ ]:
%%time

indices = []

for i in range(1,N+1):
    
    # add upper block matrix
    new_indices = indices.copy()
    
    new_indices.append(2**i-1) # add matrix element 2^(i-1)-1,2**i-1
    new_indices.append(0)      # add matrix element 2^(i-1),0
    
    # add lower block matrix
    for j in indices:
        new_indices.append(j+2**(i-1))
    indices = new_indices
    
# We have 2^N-2 rows with two entries and 2 rows with one entry each. All entries have value 0.5. 
data = np.full(2**(N+1)-2,0.5) 

# data for row i is stored at data[indptr[i]:indptr[i+1]]
# therefore we need for example for N = 4: 2^N = 16, len(data) = 30, indptr = [0,1,3,5,...,27,29,30]
indptr = [0] + list(range(1,2**(N+1)-2,2)) + [2**(N+1)-2]

# construct the sparse matrix
A = csr_matrix((data,indices,indptr))

#vals = eigs(A, k=2**(N-2), return_eigenvectors=False)

# it turns out to be difficult to really get all eigenvals for the sparse matrix. 
# Therefore better use full matrix for now, which should be fine for small size.
B = A.toarray()
v,w = np.linalg.eig(B)

np.savez_compressed("preisach_eig_N={}.dat".format(N),v=v, w=w)


Now let us plot numerics versus theory:

In [ ]:
myload = np.load("preisach_eig_N={}.dat.npz".format(N))
v, w = myload["v"], myload["w"]

In [ ]:
w[0]/w[0,0]

In [ ]:
%matplotlib inline
plt.figure()
vh =plt.hist(np.real(v),bins=np.linspace(-1,1,2000))

Z = zeros_theory(N)
x,nums = zip(*Z)
plt.plot(x, nums,"ro")
plt.yscale("log")     



In [ ]:
%matplotlib inline


vs = np.cumsum(vh[0])/2**N
plt.plot(vs,vh[1][:-1])


In [ ]:
%matplotlib inline


vs = np.cumsum(vh[0])/2**N
plt.plot(vs,vh[1][:-1])


for i in range(2, 14):
    e = sym.sin(2 * sym.pi / i) / sym.sin(sym.pi / i) /2
    #display(sym.simplify(e), sym.N(e))
    plt.plot(1-1/(2**i-1),sym.re(sym.N(e)),"bo") 
    plt.plot(1-2/(2**i-1),sym.re(sym.N(e)),"bo")

i = 5
e = sym.sin(sym.pi / i) / sym.sin(2 * sym.pi / i) /2
plt.plot(1-10/(2**i-1),sym.re(sym.N(e)),"ro") 
plt.plot(1-9/(2**i-1),sym.re(sym.N(e)),"ro")

i = 7
e = sym.sin(sym.pi / i) / sym.sin(3 * sym.pi / i) /2
plt.plot(1-42/(2**i-1),sym.re(sym.N(e)),"ro") 
plt.plot(1-41/(2**i-1),sym.re(sym.N(e)),"ro")

e = sym.sin(3 * sym.pi / i) / sym.sin(2 * sym.pi / i) /2
plt.plot(1-18/(2**i-1),sym.re(sym.N(e)),"ro") 
plt.plot(1-17/(2**i-1),sym.re(sym.N(e)),"ro")


# Study in symmetric/assymetric basis

Let us start again with the definition of $T_n$ as 
\begin{equation}
T_n = \mathbb{I} \otimes T_{n-1} + |1_n\rangle \langle 0 1_{n-1}| + |0_n\rangle \langle 1 0_{n-1}|
\end{equation}
with $T_0=0$. By definition $T_n$ operates on an $2^n$ dimensional space $V_n$, which has basis elements $|a_1 a_2 \cdots a_n \rangle$ with $a_k\in \{0,1\}$. 

We can therefore define a symmetry operation $S$ such that it operates on the basis elements in the following way
\begin{equation}
S | a_1 \cdots a_n \rangle = S | \bar{a}_1 \cdots \bar{a}_n \rangle
\end{equation}
where $\bar{a}_k = 1-a_k$.

We can thus see that 
\begin{equation}
S T_n = T_n S 
\end{equation}
and therefore we have the following invariant subspaces of $T_n$:
\begin{equation}
S_n = \{v \in V_n : Sv=v\} \qquad A_n = \{v \in V_n : Sv=-v\}
\end{equation}

Let us study the restrictions to each of these subspaces separately. We have 





# Eigenvectors



In [ ]:
def TP(N):
    I2 = sym.Matrix.eye(2)

    T = sym.Matrix.zeros(1)

    for i in range(1,N+1):
        T = TensorProduct(I2, T)
        T[2**(i)-1,2**(i-1)-1] = 1
        T[0,2**(i-1)] = 1
        
    return T / 2

for i in range(4):
    display(i,2*TP(i))

In [ ]:
x = sym.symbols("x")
def U(n,x):
    if n<0:
        return 0
    if n==0:
        return 1
    if n==1:
        return 2*x
    
    return sym.simplify(2*x * U(n-1,x) - U(n-2,x) )
    
    
def zeroU(k, n):
    if k >= n:
        raise Exception("k not smaller than n!")
    # return 
    return sym.simplify(sym.cos( sym.pi * sym.Rational(k+1,n+1) ))

In [ ]:
sym.N(U(7,zeroU(6,9)))

In [ ]:
v = sym.Matrix.zeros(3,1)
v[1] = 2
v

In [ ]:
bin(5), int(3*"1""0"+5*"1",2)

In [ ]:
bin(j),bin((-j-1)%2**5), bin((j)%2**5)

In [ ]:


def findcorners(j,n):
    corners= []
    for i in range(n):
        K = 2**(n-i-1)
        if j < K:
            continue
        corners.append(i)
        j = (-j-1) % K
        
    return corners
    
findcorners(int("00001100",2),8)
        

In [ ]:
def EV(k, n):
    if (k+1,n+2) != sym.fraction(sym.Rational(k+1,n+2)):
        raise Exception("can not handle this")
    lambda1 = zeroU(k, n+1)
    v = sym.Matrix.zeros(2**n,1)
    v.fill(1)
    for i in range(2**n):
        for c in findcorners(i,n):
            v[i] = sym.simplify(v[i]/U(n-c,lambda1))
    return v

In [ ]:
def EV2(k, n, q=n+1):
    if q>n+1 or q ==n:
        raise Exception("Wrong q")
    if (k+1,q+1) != sym.fraction(sym.Rational(k+1,q+1)):
        raise Exception("can not handle this")
    lambda1 = zeroU(k, q)
    v = sym.Matrix.zeros(2**n,1)
    for i in range(2**n):
        v[i] = 1
        for c in findcorners(i,n):
            v[i] = sym.simplify(v[i]/U(n-c,lambda1))
    return v

In [ ]:
(1,2)==sym.fraction(sym.Rational(2,4))

In [ ]:
k=0
n=2
zeroU(k,n+1), TP(n),EV(k,n), sym.simplify(TP(n)@EV(k,n)- EV(k,n)*zeroU(k,n+1))

In [ ]:
TP(4).eigenvects()[:6]